In [ ]:
# Load necessary packages
import numpy as np
import xarray as xr
import pandas as pd

In [ ]:
# Specify path experiment output
exp_class = 'CD-Paolo'
run = 'r003_UF020_SY000_EY030'
experiment_name = f'{exp_class}_{run}'
experiment_path = f'../experiments/{exp_class}/{run[0:4]}/'

# Define which time steps (in days) you want to read in
time_range = np.arange(1,621, 1)

files = []

for i in time_range:
    file = xr.open_dataset(f'{experiment_path}/{experiment_name}_{i:07.1f}.nc')
    file.coords['t'] = i
    files.append(file)

# Concatenate the files over dimension t
# if opt == 1, save the update frequency, start year and end year to the file. 
# Also, add a dimension t_year, to which year of the geometry the data belongs.
ds_tot = xr.concat(files, dim='t')
if len(run) > 4:
    ds_tot = ds_tot.assign(update_frequency = int(experiment_name[-15:-12]))
    ds_tot = ds_tot.assign(start_year = int(experiment_name[-9:-6]))
    ds_tot = ds_tot.assign(end_year = int(experiment_name[-3:]))
    ds_tot = ds_tot.assign_coords(t_year = (ds_tot.t-1)/ds_tot.update_frequency + ds_tot.start_year)

ds_tot.to_netcdf(f'{experiment_path}/AA_{run}.nc')